### Fonction qui implémente le problème maître restreint (de la génération de colonnes)

Le problème maître restreint, qui est la reformulation Dantzig-Wolfe du MIP est :

max $\sum_{c \in C_L'} w_c z_c$

tel que :
- $\sum_{c \in C_L'(i)} z_c \le 1, \forall i \in V$,
- $z \in [0,1]^{|C_L'|}$,

où $C_L' \subset C_L$.

On note $z_c=\{0,1\}=1$ si le cycle $c$ est choisi.

On réécrit le problème en le mettant sous forme de minimisation :

min $- \sum_{c \in C_L'} w_c z_c$

tel que :
- $\sum_{c \in C_L'(i)} z_c \le 1, \forall i \in V$,
- $z \in [0,1]^{|C_L'|}$,

où $C_L' \subset C_L$.

In [2]:
function node_master(node_pool)
    # Node_pool : contient tous les patterns d'un noeud
    # node_pool[i] : 1 pattern, situé à l'indice i
    
    master = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(master, "OutputFlag", 0)
    
    # Variable z
    # Le premier indice est l'indice du pattern
    z = Array{VariableRef,1}(undef,size(node_pool,1))
    for c in 1:size(node_pool,1)
        z[c]=@variable(master,lower_bound=0)
        set_name(z[c], "z_$c")
    end
    
    # Contraintes
    # Chaque paire apparait au maximum une fois dans chaque cycle/chaque pattern
    paire=@constraint(master,paire[ind in 1:P],sum(node_pool[i][ind+1]*z[i] for i in 1:size(node_pool,1))<=1)
    
    # Objectif : maximiser le nombre de transplantations
    @objective(master,Max,sum(node_pool[i][1]*z[i] for i in 1:size(node_pool,1)))
    
    # Retourne les contraintes, les variables de référence et le modèle
    return paire,z,master
end

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[2]:12